### FigureYa130SVM_step34

小丫画图出品, 2019-9-15

欢迎关注“小丫画图”公众号，同名知识星球等你加入

小丫微信: epigenomics  E-mail: figureya@126.com

作者：罗丹de情人，如有问题请联系 1870725648@qq.com

小丫编辑校验

## 第一步和第二步

先打开FigureYa130coxSVM_step12.Rmd文件，运行第一步和第二步，生成的TCGA-LIHC-survival-relevant-DMP-matrix.csv文件作为本文档的输入文件，我把这个文件上传到微云，下载链接：<https://share.weiyun.com/5GsDqAM>。

## 环境设置

下载并安装Anaconda发行版，https://www.anaconda.com/distribution/#download-section

里面已经包含了运行本文档所需的Python3、ipython、Jupyter notebook。

在终端运行以下命令来安装所需要的包：

```bash
pip install pandas
pip install sklearn
```

## 运行代码

在python里运行下面的代码。

## 第三步，REF-SVM寻找合适的特征范围

In [ ]:
import pandas as pd
from sklearn.model_selection import KFold,train_test_split
from sklearn.feature_selection import RFE
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
import matplotlib.pyplot as plt

##生成映射字典
category_mapping={
    'High_risk':3,
    'Inter_risk':2,
    'Low_risk':1
}

##以dataframe形式读入数据
differ_methyl_450_survival_df=pd.read_csv("TCGA-LIHC-survival-relevant-DMP-matrix.csv",sep=",",index_col=0)
print(differ_methyl_450_survival_df.head(20))

##转换并获取X，y值
differ_methyl_450_survival=differ_methyl_450_survival_df.values
X=differ_methyl_450_survival[:,3:]
y = differ_methyl_450_survival_df['subtype'].map(category_mapping)
y=y.astype('int')

train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=1 / 3,
                                                    random_state=3)  # 这里划分数据以1/3的来划分 训练集训练结果 测试集测试结果


params=list(range(100,len(X[1]),100))
print("params: %s " % params)

# 下面这步运行时间长，耐心等待
test_scores = []
for param in params:
    estimator = SVC(kernel='linear')
    selector = RFE(estimator, n_features_to_select=param)
    #这里没有使用KFold，直接使用cross_val_score的cv参数，代替KFold
    ##如果要使用KFold的话在这里写for循环
    test_score = cross_val_score(selector, train_X, train_y,cv=10,scoring='f1_macro') #cv：选择每次测试折数  accuracy：评价指标是准确度,可以省略使用默认值

    print(test_score.mean())
    test_scores.append(test_score.mean())

##画出特征数量与f1_macro的值，寻找合适的特征数量范围
plt.plot(params, test_scores)
plt.xlabel("Feature number")
plt.ylabel("Cross validation")
plt.show()

              ID     subtype  ch.7.135065R  cg21072795  ch.7.2184863F  \
1   TCGA-2Y-A9GS  Inter_risk      0.033270    0.120178       0.044659   
2   TCGA-2Y-A9GT  Inter_risk      0.034189    0.253894       0.052296   
3   TCGA-2Y-A9GU    Low_risk      0.089030    0.233506       0.169582   
4   TCGA-2Y-A9GV    Low_risk      0.098144    0.254983       0.143815   
5   TCGA-2Y-A9GW  Inter_risk      0.042234    0.166660       0.070577   
6   TCGA-2Y-A9GX    Low_risk      0.111627    0.359056       0.073841   
7   TCGA-2Y-A9GY  Inter_risk      0.027780    0.186898       0.039039   
8   TCGA-2Y-A9GZ  Inter_risk      0.038186    0.049369       0.059857   
9   TCGA-2Y-A9H0    Low_risk      0.027607    0.100431       0.037327   
10  TCGA-2Y-A9H1  Inter_risk      0.037704    0.140584       0.066148   
11  TCGA-2Y-A9H2  Inter_risk      0.041992    0.099425       0.054536   
12  TCGA-2Y-A9H3  Inter_risk      0.147519    0.245783       0.150861   
13  TCGA-2Y-A9H4  Inter_risk      0.050167    0.081

跑出来的部分数据

In [ ]:
# [0.64       0.58333333 0.66666667 0.70833333 0.39130435 0.59090909
#  0.68181818 0.59090909 0.72727273 0.5       ]
# [0.68       0.625      0.58333333 0.70833333 0.56521739 0.5
#  0.63636364 0.63636364 0.77272727 0.68181818]
# [0.68       0.70833333 0.625      0.625      0.47826087 0.63636364
#  0.59090909 0.59090909 0.68181818 0.63636364]
# [0.64       0.66666667 0.625      0.625      0.43478261 0.63636364
#  0.68181818 0.59090909 0.72727273 0.63636364]
# [0.64       0.70833333 0.625      0.625      0.47826087 0.68181818
#  0.63636364 0.68181818 0.63636364 0.63636364]
# [0.64       0.70833333 0.625      0.625      0.60869565 0.59090909
#  0.68181818 0.63636364 0.63636364 0.63636364]
# [0.72       0.70833333 0.625      0.625      0.56521739 0.63636364
#  0.68181818 0.63636364 0.72727273 0.59090909]
# [0.68       0.75       0.625      0.54166667 0.60869565 0.59090909
#  0.68181818 0.63636364 0.63636364 0.59090909]
# [0.68       0.75       0.625      0.54166667 0.60869565 0.59090909
#  0.68181818 0.63636364 0.63636364 0.59090909]
# [0.68       0.70833333 0.625      0.625      0.65217391 0.63636364
#  0.68181818 0.59090909 0.63636364 0.59090909]
# [0.68       0.75       0.625      0.625      0.65217391 0.59090909
#  0.68181818 0.59090909 0.59090909 0.59090909]
# [0.68       0.70833333 0.625      0.625      0.60869565 0.59090909
#  0.68181818 0.59090909 0.63636364 0.59090909]
# [0.68       0.75       0.625      0.625      0.60869565 0.59090909
#  0.68181818 0.59090909 0.63636364 0.59090909]
# [0.68       0.75       0.625      0.625      0.60869565 0.59090909
#  0.68181818 0.59090909 0.59090909 0.59090909]
# [0.72       0.75       0.58333333 0.625      0.65217391 0.59090909
#  0.68181818 0.59090909 0.59090909 0.59090909]
# [0.68       0.75       0.625      0.58333333 0.65217391 0.59090909
#  0.68181818 0.59090909 0.59090909 0.59090909]
# [0.68       0.70833333 0.625      0.625      0.65217391 0.59090909
#  0.68181818 0.59090909 0.59090909 0.59090909]
# [0.68       0.70833333 0.625      0.58333333 0.65217391 0.59090909
#  0.68181818 0.59090909 0.59090909 0.59090909]
# [0.68       0.70833333 0.625      0.54166667 0.65217391 0.59090909
#  0.68181818 0.59090909 0.59090909 0.59090909]
# [0.68       0.75       0.625      0.54166667 0.65217391 0.59090909
#  0.68181818 0.59090909 0.59090909 0.59090909]
# [0.68       0.75       0.625      0.54166667 0.65217391 0.63636364
#  0.68181818 0.59090909 0.59090909 0.59090909]


## 第四步，FW-SVM进一步细化特征数目

In [ ]:
category_mapping={
    'High_risk':3,
    'Inter_risk':2,
    'Low_risk':1
}
differ_methyl_450_survival_df=pd.read_csv("TCGA-LIHC-survival-relevant-DMP-matrix.csv",sep=",",index_col=0)
# print(differ_methyl_450_survival_df.head(20))

differ_methyl_450_survival=differ_methyl_450_survival_df.values
print(differ_methyl_450_survival[:5])

X=differ_methyl_450_survival[:,3:]
y=differ_methyl_450_survival_df['subtype'].map(category_mapping)
y=y.astype('int')
print(X[:5])
print(y[:5])

##根据REF-SVM的值，进一步细化特征数目范围
params=list(range(500, 750,10))
print("params: %s " % params)
test_scores=[]

X_train,X_test, y_train,y_test=train_test_split(X,y,test_size=1/3, random_state=3)
X_train=preprocessing.scale(X_train)
for param in params:

    estimator = SVC(kernel='linear')
    selector = RFE(estimator, n_features_to_select=param)
    test_score = cross_val_score(selector, X_train, y_train, cv=10, scoring="accuracy")
    print(test_score.mean())
    test_scores.append(test_score.mean())

plt.plot(params, test_scores)
plt.xlabel("Feature number")
plt.ylabel("Cross validation")
plt.show()

In [2]:
import IPython
print(IPython.sys_info())

!jupyter nbconvert --to html FigureYa130coxSVM_step34.ipynb

{'commit_hash': '3a0e83735',
 'commit_source': 'installation',
 'default_encoding': 'UTF-8',
 'ipython_path': '/anaconda3/lib/python3.6/site-packages/IPython',
 'ipython_version': '7.6.1',
 'os_name': 'posix',
 'platform': 'Darwin-18.7.0-x86_64-i386-64bit',
 'sys_executable': '/anaconda3/bin/python',
 'sys_platform': 'darwin',
 'sys_version': '3.6.8 |Anaconda, Inc.| (default, Dec 29 2018, 19:04:46) \n'
                '[GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]'}
[NbConvertApp] Converting notebook FigureYa130coxSVM_step34.ipynb to html
[NbConvertApp] Writing 301756 bytes to FigureYa130coxSVM_step34.html
